In [1]:
!pip install -q transformers accelerate bitsandbytes peft trl datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 77.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Local Inference on GPU 
Model page: https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

The model you are trying to use is gated. Please make sure you have access to it by visiting the model page.To run inference, either set HF_TOKEN in your environment variables/ Secrets or run the following cell to login. 🤗

In [2]:
import os
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

os.environ['HF_TOKEN'] = user_secrets.get_secret("hugging_face")

In [3]:
from huggingface_hub import login
login(new_session=False)

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

MODEL_NAME = "meta-llama/Llama-3.1-8B-Instruct"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

# Load model in 4-bit for QLoRA
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    load_in_4bit=True,              # 4-bit quantization
    device_map="auto",              # Automatically split layers to GPU
    bnb_4bit_quant_type="nf4",      # NormalFloat4 (better for QLoRA)
    bnb_4bit_use_double_quant=True, # More compression
    torch_dtype="auto"
)

# Prepare for LoRA fine-tuning
model = prepare_model_for_kbit_training(model)

# Configure LoRA
lora_config = LoraConfig(
    r=8,            # Rank
    lora_alpha=16,  # Scaling factor
    target_modules=["q_proj", "v_proj"],  # Apply LoRA to attention layers
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Wrap model with LoRA
model = get_peft_model(model, lora_config)

print("Model ready for QLoRA fine-tuning!")


2025-08-10 17:09:21.435071: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754845761.575190      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754845761.616738      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

Model ready for QLoRA fine-tuning!


In [5]:
import pandas as pd
df = pd.read_csv('/kaggle/input/mashqa-logical-dataset/mashqa_logical_qa.csv')

In [6]:
df.head()

,question,correct_combinations,incorrect_combinations,correct_answers,distractor_answers,original_context
0,Which of the following methods can indicate th...,['A doctor can identify hepatitis C by checkin...,['Hepatitis C can only be diagnosed through a ...,A doctor can identify hepatitis C by checking ...,Hepatitis C can only be diagnosed through a ph...,Your doctor could find it when he checks your ...
1,Which statements about the reliability of the ...,['Antibody tests for hepatitis C typically bec...,['A positive antibody test guarantees that a p...,Antibody tests typically become positive aroun...,A positive antibody test guarantees that a per...,They usually show up about 12 weeks after infe...
2,Which of the following methods are effective f...,"['A high-calorie, high-protein diet is essenti...",['A low-calorie diet is recommended for managi...,A well-balanced diet that is high in calories ...,A low-calorie diet is recommended for managing...,Treating Digestive Problems People with CF nee...
3,Which factors can increase the likelihood of d...,['Chemotherapy reduces the effectiveness of th...,['Regular exercise boosts immune function and ...,Chemotherapy can increase the risk of infectio...,Regular exercise strengthens the immune system...,"Because their job is to stop infections, it ma..."
4,What precautions should be taken to protect ha...,['Wearing gloves while gardening or doing hous...,['Hands should be left uncovered to allow for ...,Wearing gloves while gardening or doing housew...,Hands should be left uncovered to allow for be...,Be careful when you garden or do housework. We...


In [7]:
import random
import ast
import pandas as pd
import numpy as np


def safe_eval(val):
    if isinstance(val, list):  # already list
        return val
    if pd.isna(val):  # NaN
        return []
    if isinstance(val, str):  # string that looks like a list
        return ast.literal_eval(val)
    return []  # fallback

df['correct_combinations'] = df['correct_combinations'].apply(safe_eval)
df['incorrect_combinations'] = df['incorrect_combinations'].apply(safe_eval)


qa_pairs = []
correct_label_cycle = ['A', 'B', 'C', 'D']
label_index = 0  # To cycle through correct answer positions


for idx, row in df.iterrows():
    if len(row['correct_combinations']) == 0 or len(row['incorrect_combinations']) == 0:
        continue  # skip if missing answers
    
    question = row['question']
    correct_ans = row['correct_combinations'][0]
    
    incorrect_list = row['incorrect_combinations']
    if len(incorrect_list) >= 3:
        incorrect_ans = random.sample(incorrect_list, 3)
    else:
        incorrect_ans = random.choices(incorrect_list, k=3)
    
    # We'll fix the correct answer position according to label_index
    labels = ['A', 'B', 'C', 'D']
    correct_label = correct_label_cycle[label_index % 4]
    label_index += 1
    
    # Place correct answer in the position indicated by correct_label
    options = incorrect_ans.copy()
    options.insert(labels.index(correct_label), correct_ans)
    
    qa_pairs.append({
        'question': question,
        'A': options[0],
        'B': options[1],
        'C': options[2],
        'D': options[3],
        'correct_label': correct_label,
        'correct_answer_text': correct_ans
    })

qa_df = pd.DataFrame(qa_pairs)
print(f"Generated {len(qa_df)} QA pairs with balanced correct answer positions.")

qa_df.to_csv('mashqa_pairs.csv',index=False)

Generated 3990 QA pairs with balanced correct answer positions.


In [8]:
qa_df.head(10)

,question,A,B,C,D,correct_label,correct_answer_text
0,Which of the following methods can indicate th...,A doctor can identify hepatitis C by checking ...,Hepatitis C can only be diagnosed through a ph...,A blood test for hepatitis C cannot provide ac...,Although a blood test for hepatitis C is ineff...,A,A doctor can identify hepatitis C by checking ...
1,Which statements about the reliability of the ...,The antibody test is effective immediately aft...,Antibody tests for hepatitis C typically becom...,The antibody test measures blood sugar levels ...,A positive antibody test guarantees that a per...,B,Antibody tests for hepatitis C typically becom...
2,Which of the following methods are effective f...,Surgical procedures are the primary treatment ...,A low-calorie diet is recommended for managing...,"A high-calorie, high-protein diet is essential...",Digestive issues related to cystic fibrosis ca...,C,"A high-calorie, high-protein diet is essential..."
3,Which factors can increase the likelihood of d...,Having a strong family history of cancer impro...,While poor nutrition can increase susceptibili...,Regular exercise boosts immune function and re...,Chemotherapy reduces the effectiveness of the ...,D,Chemotherapy reduces the effectiveness of the ...
4,What precautions should be taken to protect ha...,Wearing gloves while gardening or doing housew...,Hands should be left uncovered to allow for be...,It is safe to handle all types of tools withou...,Using gloves is sufficient for protection agai...,A,Wearing gloves while gardening or doing housew...
5,Which of the following strategies can help avo...,Participating in public events to build immuni...,Avoiding close contact with individuals who ha...,Steering clear of large gatherings while also ...,Engaging in regular physical activities with l...,B,Avoiding close contact with individuals who ha...
6,What handwashing practices should individuals ...,"While it is advised to wash hands frequently, ...",Washing hands is not important while undergoin...,Hands should be washed frequently using soap a...,Hands should be washed only once a day during ...,C,Hands should be washed frequently using soap a...
7,Which of the following hygiene tips are recomm...,Avoiding showering completely during cancer tr...,It is important to take a shower or bath every...,Sharing towels and toothbrushes can reduce the...,"Taking a daily shower or bath, while paying sp...",D,"Taking a daily shower or bath, while paying sp..."
8,When should you consider seeing a rheumatologi...,Seeing a rheumatologist is advisable even if y...,Although rheumatologists have extensive traini...,It's unnecessary to consult a rheumatologist i...,You should solely rely on your regular doctor ...,A,Seeing a rheumatologist is advisable even if y...
9,What are some important considerations for eff...,Assuming that rheumatoid arthritis only affect...,Rest is crucial for individuals with rheumatoi...,Individuals often believe that excessive rest ...,Many people with rheumatoid arthritis believe ...,B,Rest is crucial for individuals with rheumatoi...


In [9]:
def zero_shot_qa(model, tokenizer, qa_dataframe):
    """
    Performs zero-shot question answering on a DataFrame of multiple-choice questions,
    tokenizing the input correctly to avoid errors.

    Args:
        model: A language model compatible with the specified prompting format.
        tokenizer: The tokenizer for the language model.
        qa_dataframe: A pandas DataFrame with columns 'question', 'A', 'B', 'C', 'D',
                      and 'correct_label'.

    Returns:
        A list of dictionaries, where each dictionary contains the original question,
        the model's predicted label, and the correct label.
    """
    results = []
    required_cols = ['question', 'A', 'B', 'C', 'D', 'correct_label']
    if not all(col in qa_dataframe.columns for col in required_cols):
        raise ValueError(f"DataFrame must contain the following columns: {required_cols}")
    
    # Define the set of valid answer choices for quick lookup
    valid_choices = {'A', 'B', 'C', 'D'}

    for idx, row in qa_dataframe.iterrows():
        predicted_label = None
        try:
            # Construct the prompt with a clear, strict instruction
            prompt = f"""Question:
{str(row['question'])}

Options:
A. {str(row['A'])}
B. {str(row['B'])}
C. {str(row['C'])}
D. {str(row['D'])}

Please respond with ONLY the single, capital letter (A, B, C, or D) that is the correct answer. Do not include any other text, punctuation, or explanation.
Answer:"""

            # 1. TOKENIZE the input prompt string
            inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
            inputs = {key: value.to(model.device) for key, value in inputs.items()}
            
            # 2. GENERATE the response using the tokenized input
            # We explicitly pass `input_ids` and `attention_mask`
            output_tokens = model.generate(
                input_ids=inputs['input_ids'],
                attention_mask=inputs['attention_mask'],
                max_new_tokens=2,
                do_sample=False
            )
            
            # 3. DECODE the output tokens back into a string
            # We slice the output to get only the newly generated tokens
            generated_token_ids = output_tokens[0][inputs['input_ids'].shape[1]:]
            generated_text = tokenizer.decode(generated_token_ids, skip_special_tokens=True)
            
            # 4. Process the generated text for a strict answer
            # Strip all whitespace and common punctuation, then get the first character
            cleaned_text = generated_text.strip().upper().replace('.', '').replace(',', '').replace(' ', '')
            
            # Validate that the first character is a valid choice
            if cleaned_text and cleaned_text[0] in valid_choices:
                predicted_label = cleaned_text[0]

        except Exception as e:
            # Catch any errors during processing and log them
            print(f"Error processing row {idx}: {e}")
        
        results.append({
            'question': str(row['question']),
            'predicted_label': predicted_label,
            'correct_label': str(row['correct_label']).strip().upper()
        })
            
    return results

    

# Usage example
results = zero_shot_qa(model,tokenizer, qa_df)
my_results = pd.DataFrame(results)
my_results.to_csv('mashqa_zeroshot_results.csv',index=False)

for res in results[:20]:
    print(f"Q: {res['question']}")
    print(f"Predicted answer: {res['predicted_label']}")
    print(f"Correct answer: {res['correct_label']}")
    print("-----")


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more de

Q: Which of the following methods can indicate the presence of hepatitis C? (Select all that apply)
Predicted answer: A
Correct answer: A
-----
Q: Which statements about the reliability of the antibody test for hepatitis C are accurate? (Select all that apply)
Predicted answer: B
Correct answer: B
-----
Q: Which of the following methods are effective for treating digestive issues related to cystic fibrosis? (Select all that apply)
Predicted answer: C
Correct answer: C
-----
Q: Which factors can increase the likelihood of developing germ infections during cancer treatment? (Select all that apply)
Predicted answer: D
Correct answer: D
-----
Q: What precautions should be taken to protect hands while gardening or doing housework during cancer treatment? (Select all that apply)
Predicted answer: A
Correct answer: A
-----
Q: Which of the following strategies can help avoid germs during cancer treatment? (Select all that apply)
Predicted answer: B
Correct answer: B
-----
Q: What handwashing p

In [10]:
my_results.head()

,question,predicted_label,correct_label
0,Which of the following methods can indicate th...,A,A
1,Which statements about the reliability of the ...,B,B
2,Which of the following methods are effective f...,C,C
3,Which factors can increase the likelihood of d...,D,D
4,What precautions should be taken to protect ha...,A,A
